In [1]:
print("hello")

hello


In [2]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Load dataset
dataset = pd.read_csv("RecipeNLG_dataset.csv")
dataset = dataset[['ingredients', 'directions']].dropna()

# Preprocess text (lowercase, remove special characters)
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9, ]", "", text)  # Remove special characters
    return text

dataset["ingredients"] = dataset["ingredients"].apply(clean_text)
dataset["directions"] = dataset["directions"].apply(clean_text)

# Combine inputs
dataset["input_text"] = "Ingredients: " + dataset["ingredients"]
dataset["target_text"] = "Directions: " + dataset["directions"]

# Drop 50% of data randomly to make it smaller
dataset = dataset.sample(frac=0.5, random_state=42).reset_index(drop=True)

# Train-validation-test split (80% train, 10% val, 10% test)
train_data, temp_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

print(f"Train size: {len(train_data)}, Val size: {len(val_data)}, Test size: {len(test_data)}")


Train size: 892456, Val size: 111557, Test size: 111558


In [11]:
# Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset["input_text"].tolist() + dataset["target_text"].tolist())

# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_data["input_text"])
train_targets = tokenizer.texts_to_sequences(train_data["target_text"])

val_sequences = tokenizer.texts_to_sequences(val_data["input_text"])
val_targets = tokenizer.texts_to_sequences(val_data["target_text"])

test_sequences = tokenizer.texts_to_sequences(test_data["input_text"])
test_targets = tokenizer.texts_to_sequences(test_data["target_text"])

# Padding sequences to ensure uniform length
max_seq_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in train_targets))

train_sequences = pad_sequences(train_sequences, maxlen=max_seq_length, padding="post")
train_targets = pad_sequences(train_targets, maxlen=max_seq_length, padding="post")

val_sequences = pad_sequences(val_sequences, maxlen=max_seq_length, padding="post")
val_targets = pad_sequences(val_targets, maxlen=max_seq_length, padding="post")

test_sequences = pad_sequences(test_sequences, maxlen=max_seq_length, padding="post")
test_targets = pad_sequences(test_targets, maxlen=max_seq_length, padding="post")

vocab_size = len(tokenizer.word_index) + 1  # Total words in vocabulary
print(f"Vocabulary Size: {vocab_size}, Max Sequence Length: {max_seq_length}")


Vocabulary Size: 216094, Max Sequence Length: 2649


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Dense

# Ensure TensorFlow version is correct
print("TensorFlow Version:", tf.__version__)

# Define parameters
vocab_size = 10000   # Adjust this based on dataset
embedding_dim = 128
sequence_length = 100

# Build model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(vocab_size, activation='softmax')  # Output should match vocab_size for text generation
])

# Call build explicitly
model.build(input_shape=(None, sequence_length))

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

# Print summary
model.summary()


TensorFlow Version: 2.18.0


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 100, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ (None, 100, 64)        │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 10000)          │       650,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,012,432 (7.68 MB)

 Trainable params: 2,012,432 (7.68 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
history = model.fit(
    train_sequences, train_targets,
    validation_data=(val_sequences, val_targets),
    batch_size=8,
    epochs=10
)

# Save the trained model
model.save("recipe_lstm_model.h5")


MemoryError: Unable to allocate 8.81 GiB for an array with shape (892456, 2649) and data type int32